In [26]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

In [5]:
# Paths
deployments_csv = r"C:\Users\sagni\Documents\Personal Files\Research\doi_10_5061_dryad_k0p2ngfhn__v20250410\ssusa_finaldeployments.csv"
clustered_excel = r"C:\Users\sagni\Documents\Personal Files\Research\doi_10_5061_dryad_k0p2ngfhn__v20250410\Clustered_Data_Updated.xlsx"

# Load data
df_deployments = pd.read_csv(deployments_csv)
df_clustered = pd.read_excel(clustered_excel)

# Extract Deployment_IDs from Excel (assuming column name is 'Deployment_ID')
deployment_ids_in_excel = df_clustered['Deployment_ID'].unique()

# Filter deployments to only those Deployment_IDs present in Excel
filtered_deployments = df_deployments[df_deployments['Deployment_ID'].isin(deployment_ids_in_excel)]

# Count distinct years per Deployment_ID
distinct_years_count = filtered_deployments.groupby('Deployment_ID')['Year'].nunique()

# Count how many Deployment_IDs have each distinct year count
count_distribution = distinct_years_count.value_counts().sort_index()

print(count_distribution)

Year
1    9508
2      46
Name: count, dtype: int64


In [6]:
# Select only Deployment_ID and Cluster_Agglo_Updated columns from clustered data
df_clustered_subset = df_clustered[['Deployment_ID', 'Cluster_Agglo_Updated']]

# Merge on Deployment_ID (inner join to keep only matching Deployment_IDs)
df_merged = df_deployments.merge(df_clustered_subset, on='Deployment_ID', how='inner')

# Count distinct years per cluster
distinct_years_per_cluster = df_merged.groupby('Cluster_Agglo_Updated')['Year'].nunique()

# Count distribution of how many clusters have a given number of distinct years
count_distribution = distinct_years_per_cluster.value_counts().sort_index()

print(count_distribution)

Year
1    86
2    30
3    17
4    22
5    43
Name: count, dtype: int64


In [11]:
# Step 1: Count records and unique years per Deployment_ID
deployment_stats = (
    df_merged.groupby('Cluster_Agglo_Updated')
    .agg(record_count=('Year', 'count'), unique_years_count=('Year', 'nunique'))
    .reset_index()
)

# Step 2: Group by unique_years_count and sum the record_count
summary = (
    deployment_stats
    .groupby('unique_years_count')['record_count']
    .sum()
    .sort_index()
)

print(summary)

unique_years_count
1    1295
2     954
3    1074
4    1459
5    4818
Name: record_count, dtype: int64


In [23]:
# ---------------------------------------------
# Step 3: Filter clusters where unique_year_count == 5
# ---------------------------------------------
clusters_with_5_years = deployment_stats[deployment_stats['unique_years_count'] >= 5]['Cluster_Agglo_Updated']

# Filter the deployment-level merged data for those clusters
filtered_deployments = df_merged[df_merged['Cluster_Agglo_Updated'].isin(clusters_with_5_years)]

# Get the Deployment_IDs from these filtered deployments
deployment_ids_with_5_years = filtered_deployments['Deployment_ID'].unique()

# ---------------------------------------------
# Step 4: Load snapshot data and filter
# ---------------------------------------------
# Read the merged snapshot dataset
df_snapshot = pd.read_csv( r"C:\Users\sagni\Documents\Personal Files\Research\doi_10_5061_dryad_k0p2ngfhn__v20250410\merged_snapshot_usa_with_label.csv")

# Filter it to only include Deployment_IDs with 5-year clusters
filtered_snapshot = df_snapshot[df_snapshot['Deployment_ID'].isin(deployment_ids_with_5_years)]
deployment_ids_with_5_years2 = df_snapshot['Deployment_ID'].unique()

# ---------------------------------------------
# Step 5: Report number of matched records
# ---------------------------------------------
print(f"\nNumber of deployments with 5 unique years - before filtering : {len(deployment_ids_with_5_years2)}")
print(f"Number of records in merged snapshot for these deployments  - before filtering : {len(df_snapshot)}")
print(f"\nNumber of deployments with 5 unique years: {len(deployment_ids_with_5_years)}")
print(f"Number of records in merged snapshot for these deployments: {len(filtered_snapshot)}")

C:\Users\sagni\AppData\Local\Temp\ipykernel_28132\3411936815.py:16: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_snapshot = pd.read_csv( r"C:\Users\sagni\Documents\Personal Files\Research\doi_10_5061_dryad_k0p2ngfhn__v20250410\merged_snapshot_usa_with_label.csv")



Number of deployments with 5 unique years - before filtering : 9554
Number of records in merged snapshot for these deployments  - before filtering : 885087

Number of deployments with 5 unique years: 4799
Number of records in merged snapshot for these deployments: 478810


In [24]:
# Select only Deployment_ID and Cluster_Agglo_Updated columns from clustered data
df_clustered_subset = df_clustered[['Deployment_ID', 'Cluster_Agglo_Updated']]
df_clustered_subset = df_clustered_subset[df_clustered_subset['Cluster_Agglo_Updated'].isin(clusters_with_5_years)]

# Merge on Deployment_ID (inner join to keep only matching Deployment_IDs)
final = df_snapshot.merge(df_clustered_subset, on='Deployment_ID', how='inner')
print(f"Number of records in final: {len(final)}")

Number of records in final: 478810


In [25]:
final.head()

,Year,Project,Camera_Trap_Array,Deployment_ID,Sequence_ID,Start_Time,End_Time,Class,Order,Family,...,Survey_Nights,Latitude,Longitude,Habitat,Development_Level,Feature_Type,x,y,cluster_label,Cluster_Agglo_Updated
0,2019,Snapshot USA 2019,Crupi,AK_Forest_Chilkat_Preserve_1,d58722s1,2019/08/31 06:50:00,2019/08/31 06:50:00,mammalia,carnivora,ursidae,...,64,59.42643,-136.2225,forest,wild,water source,-1.097526e+06,1.305164e+07,16,156
1,2019,Snapshot USA 2019,Crupi,AK_Forest_Chilkat_Preserve_1,d58722s2,2019/08/31 14:15:00,2019/08/31 14:17:00,mammalia,carnivora,ursidae,...,64,59.42643,-136.2225,forest,wild,water source,-1.097526e+06,1.305164e+07,16,156
2,2019,Snapshot USA 2019,Crupi,AK_Forest_Chilkat_Preserve_1,d58722s3,2019/08/31 18:22:00,2019/08/31 18:22:00,mammalia,carnivora,ursidae,...,64,59.42643,-136.2225,forest,wild,water source,-1.097526e+06,1.305164e+07,16,156
3,2019,Snapshot USA 2019,Crupi,AK_Forest_Chilkat_Preserve_1,d58722s4,2019/08/31 20:58:00,2019/08/31 20:58:00,mammalia,carnivora,ursidae,...,64,59.42643,-136.2225,forest,wild,water source,-1.097526e+06,1.305164e+07,16,156
4,2019,Snapshot USA 2019,Crupi,AK_Forest_Chilkat_Preserve_1,d58722s4,2019/08/31 20:58:00,2019/08/31 20:58:00,mammalia,carnivora,ursidae,...,64,59.42643,-136.2225,forest,wild,water source,-1.097526e+06,1.305164e+07,16,156


In [29]:
# Step 1: Filter and dedupe
df = final[['Year', 'Species', 'Cluster_Agglo_Updated']].drop_duplicates()

# Ensure Year is sorted as int
df['Year'] = df['Year'].astype(int)
years = sorted(df['Year'].unique())  # [2019, 2020, 2021, 2022, 2023]

# ------------------ Species Movement: Jaccard of Clusters ------------------

species_year_clusters = defaultdict(lambda: defaultdict(set))

# Build mapping: Species → Year → Set of Clusters
for _, row in df.iterrows():
    species_year_clusters[row['Species']][row['Year']].add(row['Cluster_Agglo_Updated'])

species_jaccard_scores = []

for species, year_map in species_year_clusters.items():
    for y1, y2 in zip(years, years[1:]):  # consecutive year pairs
        set1 = year_map.get(y1, set())
        set2 = year_map.get(y2, set())
        if set1 or set2:
            intersection = len(set1 & set2)
            union = len(set1 | set2)
            jaccard = intersection / union if union != 0 else None
            species_jaccard_scores.append({
                'Species': species,
                'Year1': y1,
                'Year2': y2,
                'JaccardSimilarity': jaccard
            })

df_species_movement = pd.DataFrame(species_jaccard_scores)

# ------------------ Location Movement: Jaccard of Species ------------------

cluster_year_species = defaultdict(lambda: defaultdict(set))

# Build mapping: Cluster → Year → Set of Species
for _, row in df.iterrows():
    cluster_year_species[row['Cluster_Agglo_Updated']][row['Year']].add(row['Species'])

cluster_jaccard_scores = []

for cluster, year_map in cluster_year_species.items():
    for y1, y2 in zip(years, years[1:]):  # consecutive year pairs
        set1 = year_map.get(y1, set())
        set2 = year_map.get(y2, set())
        if set1 or set2:
            intersection = len(set1 & set2)
            union = len(set1 | set2)
            jaccard = intersection / union if union != 0 else None
            cluster_jaccard_scores.append({
                'Cluster_Agglo_Updated': cluster,
                'Year1': y1,
                'Year2': y2,
                'JaccardSimilarity': jaccard
            })

df_cluster_movement = pd.DataFrame(cluster_jaccard_scores)

In [30]:
# Most moving species = lowest average Jaccard
# Insights You Can Drive
#     Low Jaccard scores for a species over time → Highly moving species
#     High Jaccard scores → Stationary or consistent habitat species
df_species_movement.groupby('Species')['JaccardSimilarity'].mean().sort_values()

Species
cryptoleucus    0.0
fuscipes        0.0
fuscescens      0.0
fuliginosus     0.0
formicivorus    0.0
               ... 
arctos          1.0
rufa            1.0
bison           1.0
tajacu          1.0
aberti          1.0
Name: JaccardSimilarity, Length: 203, dtype: float64

In [31]:
# Most stable locations = highest average Jaccard
# Same logic applies for locations:
#     Low scores = site has high species turnover
#     High scores = stable ecosystem
df_cluster_movement.groupby('Cluster_Agglo_Updated')['JaccardSimilarity'].mean().sort_values(ascending=False)

Cluster_Agglo_Updated
189    0.740712
169    0.725519
24     0.702610
176    0.700739
96     0.695887
196    0.692814
111    0.674466
131    0.669160
5      0.663140
170    0.660610
8      0.655534
19     0.654762
116    0.651382
157    0.650752
1      0.648046
37     0.639877
156    0.624632
75     0.618475
130    0.615630
42     0.610417
88     0.609525
151    0.608981
64     0.602892
147    0.602493
0      0.602273
21     0.586023
164    0.578303
56     0.576747
141    0.568865
54     0.566907
183    0.554774
125    0.552778
99     0.545122
10     0.542888
71     0.538356
79     0.525123
61     0.517757
4      0.506886
29     0.490575
91     0.484936
53     0.449924
51     0.444748
74     0.393304
Name: JaccardSimilarity, dtype: float64